In [1]:
// подключаем необходимые библиотеки
import $ivy.`org.scalanlp::breeze:1.0`
import scala.collection.mutable.ArrayBuffer
import breeze.linalg._

import $ivy.$                         

import scala.collection.mutable.ArrayBuffer

import breeze.linalg._

In [3]:
// читаем файл с данными
val dataFile = io.Source.fromFile("Dataset.csv")
var dataMatrix = DenseMatrix.zeros[Int](0,28)
dataFile.getLines.length

dataFile: io.BufferedSource = empty iterator
dataMatrix: DenseMatrix[Int] = 
res2_2: Int = 40950

In [3]:
// Пробегаемся по каждай строке файла
for (line <- dataFile.getLines.drop(1)) { 
    
    // готовим массив для дальнейшего его перегона в матрицу 
    var dataArray = Array[Int]()
    
    // делим строку на элементы и обрезаем пробелы
    for (elem <- line.split(",")map(_.trim)){
        // если значения нет, то добавляем 0
        if(elem == "") {
            dataArray = dataArray :+ 0 
        // иначе добавляем в массив значение елемента, приведенного к целому числу
        }else{
            dataArray = dataArray :+ elem.toInt
        }
    }
    if (dataArray.length == 28){
        dataMatrix = DenseMatrix.vertcat(dataMatrix, DenseMatrix(DenseVector(dataArray)))
    }
}

cmd3.sc:27: return outside method definition
    if(i > j) return
              ^Compilation Failed

: 

In [ ]:
// перегоняем получившийся массив в вектор
//var dv = DenseVector(dataArray)
// прокидываем вектор в матрицу
//var dataMatrix = DenseMatrix(dv)

//dataMatrix = dataMatrix.reshape(dataArray.length / 28, 28)